PART 1 - PREPARE THE SYSTEM BY INSTALLING NECESSARY REQUIREMENTS

In [11]:
import shutil
import os

%cd /content
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.18 triton==2.0.0 -U

!apt-get -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/ovseg/resolve/main/sam_vit_h_4b8939.pth -d /content -o sam_vit_h_4b8939.pth

dir_path = "./grounded-sam-api"
if os.path.exists(dir_path):
  print(f"{dir_path} exists already...Removing...")
  try:
    shutil.rmtree(dir_path)
    print(f"Directory {dir_path} was successfully removed.")
  except OSError as e:
    print(f"Error removing directory {dir_path}: {e}")
else:
  print(f"{dir_path} does not exist...Continuing...")

!git clone -b dev https://github.com/asanchezrodriguez/grounded-sam-api.git
%cd /content/grounded-sam-api
!pip install -q -r requirements.txt
%cd /content/grounded-sam-api/GroundingDINO
!pip install -q .
%cd /content/grounded-sam-api/segment_anything
!pip install -q .


import shlex
import subprocess
from pathlib import Path
from typing import Union

id_rsa_file = "/content/grounded-sam-api/id_rsa"
id_rsa_pub_file = "/content/grounded-sam-api/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

ssh_name = "id_rsa"
ssh_path = Path(os.path.dirname(os.getcwd())) / ssh_name
gen_key(ssh_path)

/content

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b29889|OK  |       0B/s|/content/sam_vit_h_4b8939.pth

Status Legend:
(OK):download completed.
./grounded-sam-api exists already...Removing...
Directory ./grounded-sam-api was successfully removed.
Cloning into 'grounded-sam-api'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 401 (delta 171), reused 385 (delta 158), pack-reused 0
Receiving objects: 100% (401/401), 40.38 MiB | 21.12 MiB/s, done.
Resolving deltas: 100% (171/171), done.
/content/grounded-sam-api
/content/grounded-sam-api/GroundingDINO
  Preparing metadata (setup.py) ... done
/content/grounded-sam-api/segment_anything
  Preparing metadata (setup.py) ... done


PART 2 - MOUNT THE GDRIVE TO GET ACCESS TO ORIGINAL IMAGES TO BE MASKED

In [12]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')
image_path = "/content/gdrive/My Drive/clothyai_images/jumper_black.jpeg"
if os.path.exists(image_path):
    file_size = os.path.getsize(image_path)
    print("File size:", file_size, "bytes")
else:
    print("File does not exist.")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
File size: 87090 bytes


PART 3 - DEPLOY THE API AND TUNNEL IT VIA NGROK

In [13]:
%cd /content/grounded-sam-api

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!sudo mv ngrok /usr/local/bin

!ngrok authtoken 2PLUpjKnFtNKYxtzZcBJ0aoVow6_5ZtzGehm2DYtJKTh2D1jC

import subprocess

p = subprocess.Popen("uvicorn server:app --reload --host 0.0.0.0 --port 8000", shell=True)

# Start ngrok as a subprocess
ngrok_process = subprocess.Popen(["ngrok", "http", "8000"])

# Wait for ngrok to start up and get the public URL
while True:
    try:
        ngrok_api = subprocess.check_output("curl http://localhost:4040/api/tunnels", shell=True)
        #ngrok_url = ngrok_api.decode("utf-8").split("public_url")[1].split(":")[0].split('"')[0]
        ngrok_url = ngrok_api.decode("utf-8").split("public_url")[1]
        print("ngrok tunnel:", ngrok_url)
        break
    except:
        continue

/content/grounded-sam-api
--2023-05-05 15:36:21--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  12.8MB/s    in 1.0s    

2023-05-05 15:36:23 (12.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
ngrok tunnel: ":"https://ae81-34-141-227-15.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:8000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"